# This notebook is designed for performing inference the best result.

In [1]:
import numpy as np
import pandas as pd
import math
from collections import Counter

### Read in test data

In [2]:
test_data = pd.read_csv('test.csv')
sample = pd.read_csv('submit_samples.csv')

id_list_test = sample['mac_hash']

id_dict = {m: [] for m in id_list_test}

# label_dict
label_dict_test = dict()

for data in list(test_data.values):
    loc = data[1]
    id_dict[data[0]].append(loc)

for k, v in id_dict.items():
    if len(v) != 14:
        id_dict[k] = [0] * (14 - len(v)) + v

data = np.array(list(id_dict.values()))
X_test = data

### Bert model with pretrained weight

In [3]:
from transformers import TFBertModel, BertConfig
import tensorflow as tf

config = BertConfig(vocab_size=15, hidden_size=16, num_hidden_layer=8, num_attention_heads=4,
                    intermediate_size=64, max_position_embeddings=14)
bert = TFBertModel(config)

input_seq = tf.keras.layers.Input(shape=(14,), name='input_seq', dtype='int32')
mask = tf.keras.layers.Input(shape=(14,), name='attention_mask', dtype='int32')

encoder = bert(input_seq, attention_mask=mask)[0][:,0] 

x = tf.keras.layers.Dense(32, activation='relu')(encoder)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)

model_1 = tf.keras.Model(inputs=[input_seq, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_1.compile(optimizer=optimizer, loss=loss, metrics=[acc])
model_1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_seq (InputLayer)         [(None, 14)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 14)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  40160       ['input_seq[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 14,                                            

In [4]:
model_1.load_weights('./checkpoint_Bert/Transformer_Bert')

### XLNet model with pretrained weight

In [5]:
from transformers import XLNetConfig, TFXLNetModel

config = XLNetConfig(vocab_size=15, d_model=16, n_layer=8, n_head=4, d_inner=64)

xlnet = TFXLNetModel(config)

input_seq = tf.keras.layers.Input(shape=(14,), name='input_seq', dtype='int32')
mask = tf.keras.layers.Input(shape=(14,), name='attention_mask', dtype='int32')

embeddings = xlnet(input_seq)[0]
embeddings = embeddings[:, 0]

x = tf.keras.layers.Dense(32, activation='relu')(embeddings)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)
model_2 = tf.keras.Model(inputs=[input_seq, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model_2.compile(optimizer=optimizer, loss=loss, metrics=[acc])
model_2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_seq (InputLayer)         [(None, 14)]         0           []                               
                                                                                                  
 tfxl_net_model (TFXLNetModel)  TFXLNetModelOutput(  28672       ['input_seq[0][0]']              
                                last_hidden_state=(                                               
                                None, 14, 16),                                                    
                                 mems=((14, None, 1                                               
                                6),                                                               
                                 (14, None, 16),                                            

In [6]:
model_2.load_weights('./checkpoint_XLNet/Transformer_XLNet')

### Test set inference with two different architectures

In [7]:
X_test_mask = np.ones((len(X_test),14))

y_prob_1 = model_1.predict([X_test, X_test_mask])
y_prob_2 = model_2.predict([X_test, X_test_mask])

y_prob = np.zeros(y_prob_1.shape)
for i in range(len(y_prob)):
    prob_bert = y_prob_1[i]
    prob_xlnet = y_prob_2[i]  
    if max(prob_bert) < 0.9 and max(prob_xlnet) > max(prob_bert) and np.argmax(prob_bert) != np.argmax(prob_xlnet) and max(prob_xlnet) > 0.65:
        y_prob[i] = prob_xlnet
    else:
        y_prob[i] = prob_bert

In [8]:
output = pd.DataFrame({'mac_hash':sample['mac_hash'], 'C0':y_prob[:,0], 'C1':y_prob[:,1], 'C2':y_prob[:,2], 'C3':y_prob[:,3], 'C4':y_prob[:,4]})
output

,mac_hash,C0,C1,C2,C3,C4
0,b882f1d44602a25349a08f4a0af32977,1.644878e-02,3.026091e-03,9.804910e-01,0.000034,1.014368e-08
1,86e644b498922f3a2fd0e6e1a1525de3,6.630775e-09,3.083047e-07,2.422495e-07,0.001803,9.981961e-01
2,691662b04ee08015062d901a4c5628b1,9.999843e-01,1.428103e-05,9.742126e-09,0.000001,2.203056e-12
3,52b5c510a28774237f4f118764c2ed6f,1.278072e-05,9.999846e-01,1.420636e-08,0.000003,2.989756e-08
4,9f3c995e53d109f532056b6eae29a0b5,5.603018e-05,1.705543e-02,9.828635e-01,0.000022,3.136814e-06
...,...,...,...,...,...,...
3424,5001c0dfe522886ed884c0dc4e5848df,9.999769e-01,2.151064e-05,7.950201e-09,0.000002,2.632553e-12
3425,f1c8781e701e89068fa0b8a62ecbc564,1.104203e-08,7.250758e-07,2.572627e-07,0.001593,9.984059e-01
3426,646136b402e136422466a2acd8636630,9.999788e-01,1.982054e-05,8.296314e-09,0.000001,2.537372e-12
3427,c7bea6491029f86ab3d1f0f9b599ca2c,2.082628e-04,2.273876e-06,9.997736e-01,0.000016,7.816325e-10


In [9]:
output.to_csv("output_Transformer_Best.csv", index=False)